# GLOBAL VARIABLE DEFINITION

To run the script, create the model class inside 

```
models/MODEL_NAME/model.py 
```

The module will be dynamically imported.

In [ ]:
# Set up the environment for a model training
MODEL_NAME = "Xception3d"
MODEL_PATH = "models/" + MODEL_NAME + "/"
MODEL_PATH_METRICS = MODEL_PATH + "metrics/"

# Set the data paths
DATASET = "FFPP" # Dataset name, values can be ["DFDC","FFPP"]

# IF FFPP, set the following paths for reorganize the data
ORIGIN_ROOT = "data/ff++/original_sequences/actors/c23/videos"
MANIP_ROOT= "data/ff++/manipulated_sequences/DeepFakeDetection/c23/videos"
OUTPUT_ROOT = "data/ff++/" # It will save the reorganized data here, train, val, test

# IF DFDC, set the following paths
METADATA_PATH = './data/metadata.json'

DATA_TRAIN_PATH = './data/ff++/train'
DATA_VAL_PATH = './data/ff++/val'
DATA_TEST_PATH = './data/ff++/test' # Change this to the test data path if available

# SET THE CROSS VALIDATION PARAMETERS
CROSS_VAL = False # Set to True for cross-validation
if CROSS_VAL:
    NUM_SPLITS = 5 # Number of splits for cross-validation
    
# Set the training parameters
BATCH_SIZE_TRAIN = 4
BATCH_SIZE_VAL = 4
BATCH_SIZE_TEST = 4

NUM_EPOCHS = 10 # Number of epochs to train
PATIENCE = 5  # Number of epochs to wait for improvement

# Set the training parameters
BEST_MODEL_PATH = MODEL_PATH+'best_model.pth'
LAST_MODEL_PATH = MODEL_PATH+'last_model.pth'

In [ ]:
# Check if MODEL_PATH folder exists
import os
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model directory '{MODEL_PATH}' does not exist. Please create it before running the script.")

if not os.path.exists(MODEL_PATH_METRICS):
    os.makedirs(MODEL_PATH_METRICS)
    print(f"Created directory '{MODEL_PATH_METRICS}' for metrics.")

# Imports

In [ ]:
# Standard libraries
import os
import json
import importlib
from datetime import datetime
import shutil
import glob

# Numerical and data manipulation
import numpy as np
import pandas as pd

# Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
import cv2

# Progress bar
from tqdm import tqdm

# PyTorch core
import torch
import torch.nn as nn
import torchvision

# PyTorch utilities
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.tensorboard import SummaryWriter

# Machine learning and evaluation
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)


In [ ]:
# Build the module path dynamically
module_path = f"models.{MODEL_NAME}.model"

# Import the module
model_module = importlib.import_module(module_path)

# Data

## Labels

In [ ]:
# Open metadata.json file
with open(METADATA_PATH) as f:
    metadata = json.load(f)


In [ ]:
# Convert the metadata to a pandas DataFrame
metadata_df = pd.DataFrame(metadata)

# Transpose the DataFrame
metadata_df = metadata_df.transpose()

# Reset the index
metadata_df.reset_index(inplace=True)

# Rename the column index to filename
metadata_df.rename(columns={'index': 'filename'}, inplace=True)

# Convert labels to integers.
# REAL = 0
# FAKE = 1
metadata_df['label'] = metadata_df['label'].map({'REAL': 0, 'FAKE': 1})

In [ ]:
metadata_df

## Dataset creation

In [ ]:
def extract_frames(video_path, num_frames=16):
    """Extract `num_frames` evenly spaced frames from a video."""
    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = list(np.linspace(0, length-1, num_frames).astype(int))
    frames = []
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()
    return frames

In [ ]:
class DeepfakeDataset(Dataset):
    def __init__(self, video_dir, labels_df, transform=None, num_frames=16):
        self.video_dir = video_dir
        
        # Remove all the labels that are not in the video_dir
        labels_df = labels_df[labels_df['filename'].isin(os.listdir(video_dir))]
        
        self.labels = labels_df
        self.transform = transform
        self.num_frames = num_frames

    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        video_file = self.labels.iloc[idx]['filename']
        label = self.labels.iloc[idx]['label']  # 0: real, 1: fake
        video_path = os.path.join(self.video_dir, video_file)
        frames = extract_frames(video_path, num_frames=self.num_frames)
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        # stack to (num_frames, C, H, W)
        frames = torch.stack(frames)
        return frames, torch.tensor(label, dtype=torch.long)
    
    def get_filenames(self):
        return self.labels['filename'].tolist()


In [ ]:
def prepare_ffpp_splits(
    orig_root: str,
    manip_root: str,
    output_root: str,
    val_size: float = 0.15,
    test_size: float = 0.15,
    random_state: int = 42,
    use_symlinks: bool = False,
):
    """
    1. Gather all videos from orig_root (label=0) and manip_root (label=1)
    2. Split into train/val/test with stratification on label
    3. Copy or symlink them into output_root/{train,val,test}/{real,fake}/
    """
    # 1) gather
    orig_paths = glob.glob(os.path.join(orig_root, "*.mp4"))
    manip_paths = glob.glob(os.path.join(manip_root, "*.mp4"))
    records = []
    for p in orig_paths:
        records.append({"filename": os.path.basename(p), "label": 0, "path": p})
    for p in manip_paths:
        records.append({"filename": os.path.basename(p), "label": 1, "path": p})
    df = pd.DataFrame.from_records(records)

    # 2) split
    train_df, temp_df = train_test_split(
        df, test_size=(val_size + test_size), 
        stratify=df["label"], random_state=random_state
    )
    # split temp into val/test proportional
    rel_val = val_size / (val_size + test_size)
    val_df, test_df = train_test_split(
        temp_df, test_size=(1 - rel_val),
        stratify=temp_df["label"], random_state=random_state
    )

    splits = {"train": train_df, "val": val_df, "test": test_df}

    # 3) copy or symlink
    for split_name, split_df in splits.items():
        for _, row in split_df.iterrows():
            lbl_folder = "real" if row["label"] == 0 else "fake"
            dest_dir = os.path.join(output_root, split_name, lbl_folder)
            os.makedirs(dest_dir, exist_ok=True)
            dest_path = os.path.join(dest_dir, row["filename"])
            if use_symlinks:
                if os.path.exists(dest_path):
                    os.remove(dest_path)
                os.symlink(os.path.abspath(row["path"]), dest_path)
            else:
                # copy only if not already there
                if not os.path.exists(dest_path):
                    shutil.copy2(row["path"], dest_path)

    print("Done! Splits created under", output_root)

class FFPPVideoDataset(Dataset):
    """
    Standard torch Dataset for one split of FF++:
       root/
         train/ or val/ or test/
           real/  *.mp4
           fake/  *.mp4
    """
    def __init__(self, split_root, transform=None, num_frames=16):
        """
        split_root: e.g. "data/train"
        """
        self.transform = transform
        self.num_frames = num_frames

        # find all videos and labels
        self.samples = []
        for label, sub in enumerate(["real", "fake"]):
            folder = os.path.join(split_root, sub)
            for vid in os.listdir(folder):
                if vid.lower().endswith(".mp4"):
                    self.samples.append({
                        "path": os.path.join(folder, vid),
                        "label": label
                    })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        frames = extract_frames(sample["path"], num_frames=self.num_frames)
        if self.transform:
            frames = [self.transform(f) for f in frames]
        frames = torch.stack(frames)            # (num_frames, C, H, W)
        label = torch.tensor(sample["label"], dtype=torch.long)
        return frames, label

    def get_filenames(self):
        return [os.path.basename(s["path"]) for s in self.samples]


In [ ]:
data_transforms = model_module.get_data_transform()

if DATASET == "DFDC":
    # Create datasets for training & validation, download if necessary
    training_set = DeepfakeDataset(DATA_TRAIN_PATH, metadata_df, transform=data_transforms)
    validation_set = DeepfakeDataset(DATA_VAL_PATH, metadata_df, transform=data_transforms)
    test_set = DeepfakeDataset(DATA_TEST_PATH, metadata_df, transform=data_transforms)
elif DATASET == "FFPP":
    # 1. Prepare splits (run once)
    prepare_ffpp_splits(
        orig_root=ORIGIN_ROOT,
        manip_root=MANIP_ROOT,
        output_root="/".join(DATA_TRAIN_PATH.split('/')[:-1]),
        val_size=0.15,
        test_size=0.15,
        use_symlinks=True  # or True, if you prefer
    )

    # 2. Instantiate datasets
    training_set = FFPPVideoDataset(DATA_TRAIN_PATH, transform=data_transforms)
    validation_set   = FFPPVideoDataset(DATA_VAL_PATH,   transform=data_transforms)
    test_set  = FFPPVideoDataset(DATA_TEST_PATH,  transform=data_transforms)


# Create data loaders for our datasets; shuffle for training, not for validation
# If CROSS_VAL is set to True, the loaders will be created in the cross-validation loop
if not CROSS_VAL:
    training_loader = torch.utils.data.DataLoader(training_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
    validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE_VAL, shuffle=False)

# Create data loader for test set
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)

# Class labels
classes = ('FAKE', 'REAL')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

# Visualization

In [ ]:
# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))


In [ ]:
dataiter = iter(training_loader)
images, labels = next(dataiter)

In [ ]:
images.shape, labels.shape

In [ ]:
# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid([images[0][0], images[1][0], images[2][0], images[3][0]], nrow=4)
matplotlib_imshow(img_grid)
print('  '.join(classes[labels[j]] for j in range(4)))

# Model

In [ ]:
# Now can access things inside model_module
model = model_module.Model()

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print ("Using device:", device)

# Multi-GPU support
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    
model = model.to(device)

# Loss function

In [ ]:
loss_fn = model_module.get_loss_fn()

# Optimizer

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = model_module.get_optimizer(model.parameters())
scheduler = None

if isinstance(optimizer, tuple):
    optimizer, scheduler = optimizer
    

# Save all the params

In [ ]:
def serialize_value(value):
    """Helper to safely serialize values for JSON."""
    if isinstance(value, torch.Tensor):
        return value.tolist()
    elif isinstance(value, dict):
        return {k: serialize_value(v) for k, v in value.items()}
    elif isinstance(value, set):
        return list(value)
    else:
        return str(value)

def extract_params(obj):
    """Extracts public parameters from an object's __dict__, serializing them."""
    params = {}
    for key, value in getattr(obj, '__dict__', {}).items():
        if key.startswith('_'):
            continue  # skip private attributes
        params[key] = serialize_value(value)
    return params

def get_transform_info(transform):
    """Extracts transform name and parameters."""
    return {
        'name': transform.__class__.__name__,
        'params': extract_params(transform)
    }

# --- LOSS CONFIG ---
loss_config = extract_params(loss_fn)

# --- OPTIMIZER CONFIG ---
optimizer_params = serialize_value(optimizer.defaults)

# --- SCHEDULER CONFIG ---
if scheduler is not None:
    scheduler_params = serialize_value(scheduler.__dict__)
    scheduler_config = {
        'scheduler': scheduler.__class__.__name__,
        'params': scheduler_params
    }
else:
    scheduler_config = None

# --- DATA TRANSFORMS CONFIG ---
if hasattr(data_transforms, 'transforms'):
    transform_list = [get_transform_info(t) for t in data_transforms.transforms]
else:
    transform_list = [get_transform_info(data_transforms)]

# --- FINAL MODEL INFO ---
model_info = {
    'model': MODEL_NAME,
    'optimizer': optimizer.__class__.__name__,
    'optimizer_params': optimizer_params,
    'scheduler': scheduler_config,
    'scheduler_params': scheduler_params if scheduler is not None else None,
    'loss_fn': loss_fn.__class__.__name__,
    'loss_fn_params': loss_config,
    'data_transforms': transform_list,
    'dataset': DATASET,
    'training_set_size': len(training_set),
    'validation_set_size': len(validation_set),
    'test_set_size': len(test_set),
    'batch_size_train': BATCH_SIZE_TRAIN,
    'batch_size_val': BATCH_SIZE_VAL,
    'batch_size_test': BATCH_SIZE_TEST,
    'num_epochs': NUM_EPOCHS,
    'patience': PATIENCE,
    'best_model_path': BEST_MODEL_PATH,
    'last_model_path': LAST_MODEL_PATH,
    'cross_val': CROSS_VAL,
    'num_splits': NUM_SPLITS if CROSS_VAL else None,
    'date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# --- SAVE JSON ---
os.makedirs(MODEL_PATH, exist_ok=True)
with open(os.path.join(MODEL_PATH, 'model_info.json'), 'w') as f:
    json.dump(model_info, f, indent=4)

print("Model info saved successfully ✅")


# Training loop

In [ ]:
# Function to train the model for one epoch
def train_epoch(model, loader, criterion, optimizer, writer, epoch):
    """
    Train the model for one epoch.

    Args:
        model: The model to train.
        loader: DataLoader for the training data.
        criterion: Loss function.
        optimizer: Optimizer for updating model weights.
        writer: TensorBoard SummaryWriter for logging.
        epoch: Current epoch number.

    Returns:
        epoch_loss: Average loss for the epoch.
        epoch_acc: Accuracy for the epoch.
    """
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc=f"Training Epoch {epoch+1}"):
        # Move inputs and labels to the device (GPU/CPU)
        inputs, labels = inputs.to(device), labels.to(device)
    
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels.float().unsqueeze(1)) # Ensure labels are of shape (batch_size, 1)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).long().squeeze(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
    # Scheduler step if using a scheduler
    if scheduler is not None:
        scheduler.step()
        print(f"Learning rate: {scheduler.get_last_lr()[0]}")

    # Calculate average loss and accuracy
    epoch_loss = running_loss / total
    epoch_acc = correct / total

    # Log training loss and accuracy to TensorBoard
    writer.add_scalar('Training Loss', epoch_loss, epoch)
    writer.add_scalar('Training Accuracy', epoch_acc, epoch)

    return epoch_loss, epoch_acc

# Function to evaluate the model for one epoch
def eval_epoch(model, loader, criterion, writer, epoch):
    """
    Evaluate the model for one epoch.

    Args:
        model: The model to evaluate.
        loader: DataLoader for the validation data.
        criterion: Loss function.
        writer: TensorBoard SummaryWriter for logging.
        epoch: Current epoch number.

    Returns:
        epoch_loss: Average loss for the epoch.
        epoch_acc: Accuracy for the epoch.
    """
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in tqdm(loader, desc=f"Validation Epoch {epoch+1}"):
            # Move inputs and labels to the device (GPU/CPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels.float().unsqueeze(1)) # Ensure labels are of shape (batch_size, 1)

            # Accumulate loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long().squeeze(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    # Calculate average loss and accuracy
    epoch_loss = running_loss / total
    epoch_acc = correct / total

    # Log validation loss and accuracy to TensorBoard
    writer.add_scalar('Validation Loss', epoch_loss, epoch)
    writer.add_scalar('Validation Accuracy', epoch_acc, epoch)

    return epoch_loss, epoch_acc


# Per-epoch activity

In [ ]:
# Check if the model already exists
if os.path.exists(BEST_MODEL_PATH):
    # Ask the user if they want to overwrite the existing model
    user_input = input(f"Model already exists at {BEST_MODEL_PATH}. Do you want to overwrite it? (y/n): ")
    if user_input.lower() == 'y':
        pass
    else:
        raise ValueError(f"Model already exists at {BEST_MODEL_PATH}. Please remove it before training.")

In [ ]:
def per_epoch_activity(num_epochs, training_loader, validation_loader, model, loss_fn, optimizer, writer, patience=5):
    """
    Train and evaluate the model for a specified number of epochs.
    
    Args:
        num_epochs: Number of epochs to train.
        training_loader: DataLoader for the training data.
        validation_loader: DataLoader for the validation data.
        model: The model to train.
        loss_fn: Loss function.
        optimizer: Optimizer for updating model weights.
        writer: TensorBoard SummaryWriter for logging.
        patience: Number of epochs to wait for improvement before stopping.
    """
    best_acc = 0.0 # Best validation accuracy
    no_improve_epochs = 0  # Counter for epochs without improvement
    
    for epoch in tqdm(range(num_epochs)):
        train_loss, train_acc = train_epoch(model, training_loader, loss_fn, optimizer, writer, epoch)
        val_loss, val_acc = eval_epoch(model, validation_loader, loss_fn, writer, epoch)
        print(f"Epoch {epoch+1}/{num_epochs}", 
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}", 
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            no_improve_epochs = 0  # Reset counter if validation accuracy improves
        else:
            no_improve_epochs += 1

        # Early stopping
        if no_improve_epochs >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        # Save the model last epoch
        torch.save(model.state_dict(), LAST_MODEL_PATH)

In [ ]:

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(MODEL_PATH+'/runs/deepfake_logs_{}'.format(timestamp))

if CROSS_VAL:
    # K-Fold Cross Validation
    splits = NUM_SPLITS
    kfold = KFold(n_splits=splits, shuffle=True)
    dataset = ConcatDataset([training_set, validation_set])

    for fold, (train_ids, test_ids) in tqdm(enumerate(kfold.split(dataset))):

        print(f'Fold {fold+1}/{splits}')
        training_loader = torch.utils.data.DataLoader(training_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
        validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE_VAL, shuffle=False)
            
        per_epoch_activity(NUM_EPOCHS, training_loader, validation_loader, model, loss_fn, optimizer, writer, patience=PATIENCE)
else:
    # No K-Fold Cross Validation
    per_epoch_activity(NUM_EPOCHS, training_loader, validation_loader, model, loss_fn, optimizer, writer, patience=PATIENCE)
    
# Close the TensorBoard writer
writer.close()

# Open saved model

In [ ]:
saved_model = model_module.Model()
saved_model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Inference

In [ ]:
# Example inference on a single video
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

video_path = 'data/val_sub/avoqheikrk.mp4'
frames = extract_frames(video_path, num_frames=16)
inputs = data_transforms(frames[0]).unsqueeze(0)  # For demonstration, use first frame
inputs = inputs.to(device)
with torch.no_grad():
    output = model(inputs.unsqueeze(1))  # shape (1, num_frames, C, H, W)
    pred = output.argmax(dim=1).item()
print("Predicted label: ", 'Fake' if pred == 1 else 'Real')
print("Real label: ", 'Fake' if metadata_df[metadata_df['filename'] == 'acagallncj.mp4']['label'].values[0]== 1 else 'Real')

# Test

In [ ]:
# Run test loop on validation set
model.eval()
all_labels, all_preds = [], []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Compute metrics
acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='binary')
rec  = recall_score(all_labels, all_preds, average='binary')
f1   = f1_score(all_labels, all_preds, average='binary')

class_report = classification_report(all_labels, all_preds, labels=classes, target_names=classes)

In [ ]:
print(f"Test Accuracy : {acc:.4f}")
print(f"Precision     : {prec:.4f}")
print(f"Recall        : {rec:.4f}")
print(f"F1 Score      : {f1:.4f}\n")
print("Classification Report:\n", class_report)


In [ ]:
# Save results to CSV
results_df = pd.DataFrame({
    'filename': test_set.get_filenames(),
    'label': all_labels,
    'predicted_label': all_preds
})
results_df['correct'] = results_df['label'] == results_df['predicted_label']
results_df.to_csv(MODEL_PATH_METRICS+'results.csv', index=False)

In [ ]:
# Save Acc, Precision, Recall, F1 to CSV
metrics_df = pd.DataFrame({
    'accuracy': [acc],
    'precision': [prec],
    'recall': [rec],
    'f1_score': [f1]
})
metrics_df.to_csv(MODEL_PATH_METRICS+'metrics.csv', index=False)

# Save classification report to txt file
with open(MODEL_PATH_METRICS+'classification_report.txt', 'w') as f:
    f.write(class_report)


In [ ]:
# Plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

plt.savefig(MODEL_PATH_METRICS+'confusion_matrix.png')
plt.show()